In [3]:
import numpy as np
import random as random
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import extended_model as em

ModuleNotFoundError: No module named 'extended_model'

In [5]:
def generate_city_grid(n, branch_prob, road_length):
    grid = np.ones((n, n), dtype=int)

    # Select from a central half the "main" roads
    horizontal_road = random.randint(n // 4, 3 * n // 4)
    vertical_road = random.randint(n // 4, 3 * n // 4)
    
    # Set "main" roads
    grid[horizontal_road, :] = -1
    grid[:, vertical_road] = -1

    def grow_road(x, y, direction, length):
        for a in range(length):
            if 0 <= x < n and 0 <= y < n:
                grid[x, y] = -1
                # Chance to snake off in a random direction
                if random.random() < 0.3:
                    #Represents the directions, 0 being no change in that direction, 1 being to the right / down and -1 being to the left / up
                    direction = random.choice([(0, 1), (1, 0), (0, -1), (-1, 0)])
                    # Adds onto the road for both our x and y direcrtions
                x += direction[0]
                y += direction[1]
    
    # Generate branches along the primary roads with snaky paths
    for i in range(n):
        for j in range(n):
            # Find a tile with road on it and use our probability
            if grid[i, j] == -1 and random.random() < branch_prob:
                # 
                initial_direction = random.choice([(0, 1), (1, 0), (0, -1), (-1, 0)])  # Right, Down, Left, Up
                grow_road(i, j, initial_direction, road_length)
                
    for i in range(1, n - 1): # Use n-1 as receiving indexing error, possibly due to bounding
        for j in range(1, n - 1):
            if grid[i, j] == -1:
                # Check if all surrounding tiles are road
                # This is annoying as it can only get the 1 from the left to right and top to bottom first,
                # Then the others don't count as they have 1 house neighbour
                if (grid[i-1, j] == -1 and grid[i+1, j] == -1 and
                    grid[i, j-1] == -1 and grid[i, j+1] == -1):
                    grid[i, j] == 1  #Change out of road

    return grid

n = 50
city_grid = generate_city_grid(n, branch_prob=0.3, road_length= n//5)

print(city_grid)


[[ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 ...
 [-1 -1 -1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]]


In [ ]:
plt.imshow(city_grid, cmap="gray")
plt.title("City Grid")
plt.show()

In [4]:
def initialize_affluence_and_house_grids(city_grid, affluence_levels, p):
    """
    Initializes affluence and house grids based on the generated city grid.
    Houses receive random affluence levels, while roads remain at 0.
    """
    rho = 0.7
    n, m = city_grid.shape
    noise = np.random.rand(n, m)
    affluence_grid = np.zeros((n, m), dtype=float)
    house_grid = np.zeros((n, m), dtype=float)
    politics_grid = np.zeros((n, m), dtype = float)
    religion_grid = np.zeros((n, m), dtype = float)
    ethnicity_grid = np.zeros((n, m), dtype = float)
    for i in range(n):
        for j in range(m):
            if city_grid[i, j] == 1:  # 1 is our value for houses
                affluence_grid[i, j] = np.random.choice(
                    [affluence_levels["rich"], affluence_levels["middle"], affluence_levels["poor"]],
                    p=p
                )
                politics_grid[i,j] = rho * affluence_grid[i,j] + (1- rho) * noise[i,j]
                religion_grid[i,j] = np.random.choice([1,0])
                ethnicity_grid[i, j] = np.random.choice([1,2,3,4,5,6])
                house_grid[i, j] = np.random.uniform(0,1)  # Mark as a house
            elif city_grid[i, j] == -1: 
                affluence_grid[i, j] = -1  # give as road in affluence grid
                house_grid[i,j] = -1
                politics_grid[i,j] = -1
                religion_grid[i,j] = -1
                ethnicity_grid[i,j] = -1

    return affluence_grid, house_grid, politics_grid, religion_grid, ethnicity_grid


In [ ]:
def calculate_neighborhood_average(house_grid, x, y, r):
    """
    Calculate the average house value in the square neighborhood of radius r around (x, y).
    """
    n, m = house_grid.shape
    weighted_sum = 0
    total_weight = 0
    x_min, y_min = max(0,x-r), max(0,y-r)
    x_max, y_max = min(n,x+r), min(m,y+r)
    
    # Iterate over the neighborhood within the radius
    for i in range(x_min, x_max):
        for j in range(y_min, y_max):
            if house_grid[x, y] == -1:
                break
            if house_grid[i, j] >= 0:
                # Calculate the distance from (x, y)
                dx, dy = x - i, y - j
                distance_squared = dx**2 + dy**2
                # Avoid division by zero by setting a minimum distance
                if distance_squared == 0:
                    weight = 1
                else:
                    weight = 1 / distance_squared
                weighted_sum += house_grid[i, j] * weight
                total_weight += weight

    # Check if total_weight is zero to avoid division by zero
    if total_weight == 0:
        return 0  # or return None, depending on your requirements
    # Calculate the weighted average of the neighborhood
    weighted_average = weighted_sum / total_weight
    return weighted_average

In [ ]:
def calculate_ethnicity_average(ethnicity_grid, x, y, r):
    n, m = ethnicity_grid.shape
    x_min, y_min = max(0,x-r), max(0,y-r)
    x_max, y_max = min(n,x+r), min(m,y+r)
    count = 0
    total = 1
    for i in range(x_min, x_max):
        for j in range(y_min, y_max):
            if ethnicity_grid[x, y] == -1:
                break
            if ethnicity_grid[i,j] >= 0:
                if ethnicity_grid[i,j] == ethnicity_grid[x,y]:
                    count += 1
                    total += 1
                else:
                    total +=1
    if total == 1:
        return 1
    else:
        return count / total

In [ ]:
def update_values(affluence_grid, house_grid, lambda_):
    """
    Update house values based on the affluence and neighborhood average.
    """
    new_house_grid = np.copy(house_grid)
    n, m = house_grid.shape
    for x in range(n):
        for y in range(m):
            # Added if statement to stop roads from being swapped
            if affluence_grid[x, y] >= 0 and house_grid[x, y] >= 0:
                affluence = affluence_grid[x, y]
                neighborhood_avg = calculate_neighborhood_average(house_grid, x, y, r)
                new_house_grid[x, y] = affluence + lambda_ * neighborhood_avg
    return new_house_grid

In [ ]:
def propose_swap(affluence_grid, house_grid, politics_grid, religion_grid, ethnicity_grid):
    """
    Calculate delta and swap agents if delta > 0.
    """
    pol_weight = 1/3
    rel_weight = 1/3
    eth_weight = 1/3
    n, m = house_grid.shape
    # Choose two random houses
    x1, y1 = random.randint(0, n-1), random.randint(0, m-1)
    x2, y2 = random.randint(0, n-1), random.randint(0, m-1)

    A_x1, V_x1 = affluence_grid[x1, y1], house_grid[x1, y1]
    A_x2, V_x2 = affluence_grid[x2, y2], house_grid[x2, y2]
    delta_econ = (A_x1 - V_x1)**2 + (A_x2 - V_x2)**2 - (A_x1 - V_x2)**2 - (A_x2 - V_x1)**2
    
    P_x1, P_x2 = politics_grid[x1, y1], politics_grid[x2, y2]
    P_nbhd_x1, P_nbhd_x2 = calculate_neighborhood_average(politics_grid, x1, y1, r), calculate_neighborhood_average(politics_grid, x2, y2, r)
    delta_pol = (P_x1 - P_nbhd_x1)**2 + (P_x2 - P_nbhd_x2)**2 - (P_x1 - P_nbhd_x2)**2 - (P_x2 - P_nbhd_x1)**2 

    R_x1, R_x2 = religion_grid[x1, y1], religion_grid[x2, y2]
    R_nbhd_x1, R_nbhd_x2 = calculate_neighborhood_average(religion_grid, x1, y1, r), calculate_neighborhood_average(religion_grid, x2, y2, r)
    delta_rel = (R_x1 - R_nbhd_x1)**2 + (R_x2 - R_nbhd_x2)**2 - (R_x1 - R_nbhd_x2)**2 - (R_x2 - R_nbhd_x1)**2

    delta_eth = em.calculate_ethnic_delta(ethnicity_grid, x1, y1, x2, y2, r)
    
    if delta_econ + rel_weight * delta_rel + pol_weight * delta_pol + eth_weight * delta_eth > 0:
        # Swap the two agents on all grids
        affluence_grid[x1, y1], affluence_grid[x2, y2] = A_x2, A_x1
        house_grid[x1, y1], house_grid[x2, y2] = V_x2, V_x1
        politics_grid[x1, y1], politics_grid[x2, y2] = P_x2, P_x1
        religion_grid[x1, y1], religion_grid[x2, y2] = R_x2, R_x1
        ethnicity_grid[x1, y1], ethnicity_grid[x2, y2] = ethnicity_grid[x2, y2], ethnicity_grid[x1, y1]
        # Swap the affluence levels of the two agents
        affluence_grid[x1, y1], affluence_grid[x2, y2] = A_x2, A_x1
        house_grid[x1, y1], house_grid[x2, y2] = V_x2, V_x1

In [ ]:
def plot_grid(house_grid, step):
    """
    Plot the current state of the grid.
    """
    plt.imshow(house_grid, cmap='plasma', origin='upper')
    plt.title(f'Step {step}')
    plt.colorbar(label='House Value')
    plt.show()
    plt.pause(0.5)
    clear_output(wait=True)
    display(plt.gcf())

In [ ]:
n = 50
lambda_ = 0.8
r = 2
branch_prob = 0.3
road_length = n//8
p = [0.1, 0.6, 0.3]
affluence_levels = {
    "rich": 0.7,
    "middle": 0.5,
    "poor": 0.3
}
city_grid = generate_city_grid(n, branch_prob, road_length)

In [2]:
# Simulation parameters
n_steps = 200
affluence_grid, house_grid, politics_grid, religion_grid, ethnicity_grid = initialize_affluence_and_house_grids(city_grid, affluence_levels, p)

fig, ax = plt.subplots()
fig.suptitle('Step 0')
img = plt.imshow(house_grid, cmap='viridis',vmin=0, vmax=3.5)

cbar = fig.colorbar(img, ax=ax)
cbar.set_label('House Price')


# Simulation loop
for step in range(n_steps):
    # Update house values
    house_grid = update_values(affluence_grid, house_grid, lambda_)
    
    # Propose random move - several in each step
    for i in range(1000):
        propose_swap(affluence_grid, house_grid, politics_grid, religion_grid, ethnicity_grid)

    img.set_array(house_grid)
    img.set_clim(vmax = np.max(house_grid))
    fig.suptitle(f"Step {step}")
    plt.pause(0.001)
    clear_output(wait=True)
    display(fig)
    

NameError: name 'initialize_affluence_and_house_grids' is not defined

In [ ]:
img.set_array(politics_grid)
img.set_clim(vmax = np.max(politics_grid))
cbar.set_label('Political View')
display(fig)
img.set_array(affluence_grid)
img.set_clim(vmin = np.min(affluence_grid), vmax = np.max(affluence_grid))
cbar.set_label('Affluence')
display(fig)
img.set_array(house_grid)
img.set_clim(vmax = np.max(house_grid))
cbar.set_label('House Price')
display(fig)
img.set_array(religion_grid)
img.set_clim(vmin = -1, vmax = np.max(religion_grid))
cbar.set_label('Religious (1,0)')
display(fig)
img.set_array(ethnicity_grid)
img.set_clim(vmin =-1, vmax = np.max(ethnicity_grid))
cbar.set_label('Ethnicites')
display(fig)

In [ ]:
house_grid.mean()